In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("../data/02_intermediate/merged_data.csv")


In [8]:
df.columns


Index(['origin', 'destination', 'departure_date', 'query_date',
       'days_before_departure', 'airline', 'price', 'currency', 'stops',
       'flight_duration', 'cabin', 'offer_rank', 'departure_time',
       'arrival_time', 'source'],
      dtype='object')

In [9]:
df

,origin,destination,departure_date,query_date,days_before_departure,airline,price,currency,stops,flight_duration,cabin,offer_rank,departure_time,arrival_time,source
0,CDG,AMS,2026-01-15,2026-01-03,12,Air France,117.0,EUR,1,495,ECONOMY,1,"9:20 AM on Thu, Jan 15","5:35 PM on Thu, Jan 15",google_flights
1,CDG,AMS,2026-01-15,2026-01-03,12,Norwegian,119.0,EUR,1,380,ECONOMY,2,"12:35 PM on Thu, Jan 15","6:55 PM on Thu, Jan 15",google_flights
2,CDG,AMS,2026-01-15,2026-01-03,12,Air France,125.0,EUR,1,375,ECONOMY,3,"9:20 AM on Thu, Jan 15","3:35 PM on Thu, Jan 15",google_flights
3,CDG,AMS,2026-01-15,2026-01-03,12,Air France,117.0,EUR,1,635,ECONOMY,4,"7:00 AM on Thu, Jan 15","5:35 PM on Thu, Jan 15",google_flights
4,CDG,AMS,2026-01-16,2026-01-03,13,Norwegian,127.0,EUR,1,305,ECONOMY,1,"12:30 PM on Fri, Jan 16","5:35 PM on Fri, Jan 16",google_flights
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91205,FCO,CDG,2026-05-31,2026-01-21,130,ITA,99.0,EUR,0,130,ECONOMY,1,"9:30 PM on Sun, May 31","11:40 PM on Sun, May 31",google_flights
91206,FCO,CDG,2026-05-31,2026-01-21,130,ITA,129.0,EUR,0,130,ECONOMY,2,"3:10 PM on Sun, May 31","5:20 PM on Sun, May 31",google_flights
91207,FCO,CDG,2026-05-31,2026-01-21,130,ITA,145.0,EUR,0,130,ECONOMY,3,"8:35 AM on Sun, May 31","10:45 AM on Sun, May 31",google_flights
91208,FCO,CDG,2026-05-31,2026-01-21,130,ITA,145.0,EUR,0,135,ECONOMY,4,"11:00 AM on Sun, May 31","1:15 PM on Sun, May 31",google_flights


In [11]:
od_airline_dbd = (
    df
    .groupby(["origin",'destination', "airline", "days_before_departure"]),
    .agg(mean_price=("price", "mean")),
    sort_values("mean_price", ascending=False),
)

SyntaxError: invalid syntax (3831532963.py, line 4)

In [29]:
od_airline_dbd = (
    df
    .groupby(["origin", "destination", "airline", "days_before_departure"])
    .agg(mean_price=("price", "mean"))
    .sort_values("days_before_departure", ascending=True)
    .reset_index()  # optional, to turn index back into columns
)
od_airline_dbd.head(15)

,origin,destination,airline,days_before_departure,mean_price
0,CDG,LHR,Austrian,1,229.000000
1,CDG,LHR,KM Malta Airlines,1,141.000000
2,ATH,CDG,ITA,1,171.000000
3,CDG,LHR,British Airways,1,171.692308
4,CDG,LHR,Air France,1,217.000000
5,CDG,LHR,Aegean,1,154.500000
6,MAD,CDG,Iberia,1,128.076923
7,CDG,FCO,Luxair,1,300.000000
8,CDG,FCO,KM Malta Airlines,1,127.428571
9,CDG,FCO,Iberia,1,154.000000


In [60]:
db_target_test = od_airline_dbd[
    (od_airline_dbd["origin"] == "MAD") &
    (od_airline_dbd["destination"] == "CDG") &
    (od_airline_dbd["airline"] == "KLM")
]
db_target_test.head(12)

,origin,destination,airline,days_before_departure,mean_price
12,MAD,CDG,KLM,1,211.000000
290,MAD,CDG,KLM,5,180.000000
317,MAD,CDG,KLM,6,180.000000
414,MAD,CDG,KLM,7,180.000000
436,MAD,CDG,KLM,8,117.000000
577,MAD,CDG,KLM,10,104.000000
612,MAD,CDG,KLM,11,102.333333
684,MAD,CDG,KLM,12,95.000000
769,MAD,CDG,KLM,13,106.000000
825,MAD,CDG,KLM,14,95.000000


In [47]:
def extract_mean_price_at_od_airline_dbd(df_target, dbd):
    df_copy = df_target.copy()  # make a copy
    df_copy['distance_to_dbd'] = abs(df_copy["days_before_departure"] - dbd)
    closest_row = df_copy.loc[df_copy['distance_to_dbd'].idxmin()]
    return closest_row['mean_price']

In [55]:
def extract_mean_price_at_od_airline_dbd(df_target, dbd):
    # Compute absolute distance to target DBD
    distance_to_dbd = abs(df_target["days_before_departure"] - dbd)
    
    # Find the minimum distance
    min_distance = distance_to_dbd.min()
    
    # Select all rows with this minimum distance
    closest_rows = df_target[distance_to_dbd == min_distance]
    
    # Return the mean of mean_price for tied rows
    return closest_rows['mean_price'].mean()

In [61]:
extract_mean_price_at_od_airline_dbd(db_target_test,9)

110.5